In [17]:
import pandas as pd
import spacy
import re
import time
from tqdm import tqdm

In [18]:
nlp = spacy.load("de_core_news_sm", disable=["ner", "lemmatizer", "transformer"])

In [19]:
def spacy_sent_tokenize(text):
    doc = nlp(text)
    return [sent.text for sent in doc.sents]


In [20]:
file_path = "speeches.csv"
df = pd.read_csv(file_path)
text_speeches = df[~df['Interjection']]
text_speeches

,Filename,Period,Date,Item,Speech #,Paragraph #,Speaker,Role,Gender,Party,...,Interjector Gender,Interjector Party,Verbal interjection,Nonverbal interjection,Interjection type,next_verbal_interjection,next_nonverbal_interjection,next_interjectors,next_interjections,Speaker Paragraph
0,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,1,Carsten Schneider,mp,männlich,SPD,...,NaN,NaN,False,False,NaN,False,False,[],NaN,NaN
1,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,2,Carsten Schneider,mp,männlich,SPD,...,NaN,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN
5,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,3,Carsten Schneider,mp,männlich,SPD,...,NaN,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN
9,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,4,Carsten Schneider,mp,männlich,SPD,...,NaN,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN
13,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,5,Carsten Schneider,mp,männlich,SPD,...,NaN,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'CDU/CSU', 'Lachen', nan)]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550753,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,3,Florian Toncar,mp,männlich,FDP,...,NaN,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN
550755,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,4,Florian Toncar,mp,männlich,FDP,...,NaN,NaN,False,False,NaN,False,False,[],NaN,NaN
550756,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,5,Florian Toncar,mp,männlich,FDP,...,NaN,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN
550758,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,6,Florian Toncar,mp,männlich,FDP,...,NaN,NaN,False,False,NaN,False,False,[],NaN,NaN


In [21]:
party_aliases = {
    "AfD": ["AfD"],
    "CDU/CSU": ["CDU", "CSU", "CDU/CSU", "Unionspartei", "Unionsparteien", "Unionsfraktion", "Unionsfraktionen"],
    "FDP": ["FDP", "Liberalen", "Gelben", "Gelbe", "Liberale"],
    "SPD": ["SPD", "Sozialdemokraten", "Sozen", "Sozis", "Roten", "Rote"],
    "GRUENE": ["Ökopartei", "Grünen", "Grüne"],
    "DIE LINKE": ["Linke", "Linken", "Linkspartei"],
    "Government": ["Groko", "Regierungsparteien", "Bundesregierung", "Koalitionspartei", "Koalitionsparteien", "Koalitionsfraktion", "Koalitionsfraktionen"],
    "Opposition": ["Opposition", "Oppositionspartei", "Oppositionsparteien", "Oppositionsfraktion", "Oppositionsfraktionen"],
}


In [22]:
def extract_parties(paragraph, party_aliases):
    paragraph_lower = paragraph.lower()
    parties_mentioned = set()

    exclusion_pattern = r'\beuropäische(n)?\s+union\b'

    if re.search(r'\bunion\b', paragraph_lower, re.IGNORECASE) and not re.search(exclusion_pattern, paragraph_lower, re.IGNORECASE):
        parties_mentioned.add("CDU/CSU")

    if re.search(r'\bFreie\s+Demokratische\s+Partei\b', paragraph_lower, re.IGNORECASE):
        parties_mentioned.add("FDP")
    if re.search(r'\bAlternative\s+für\s+Deutschland\b', paragraph_lower, re.IGNORECASE):
        parties_mentioned.add("AfD")
    if re.search(r'\bBündnis\s+90\b|\bBündnis\s+90/Die\s+Grünen\b', paragraph_lower, re.IGNORECASE):
        parties_mentioned.add("GRUENE")
    if re.search(r'\bSozialdemokratische\s+Partei\s+Deutschlands\b', paragraph_lower, re.IGNORECASE):
        parties_mentioned.add("SPD")
    if re.search(r'\bGroße\s+Koalition\b', paragraph_lower, re.IGNORECASE):
        parties_mentioned.add("Government")

    doc = nlp(paragraph)

    for party, aliases in party_aliases.items():
        for alias in aliases:
            pattern = r'\b' + re.escape(alias.lower()) + r'\b'
            for token in doc:
                if re.search(pattern, token.text.lower()) and (token.pos_ == 'NOUN' or token.pos_ == 'PROPN'):
                    parties_mentioned.add(party)

    return list(parties_mentioned)

GET ALL DEBATES (PARAGRAPHS)

In [23]:
text_speeches['Mentioned parties'] = text_speeches['Paragraph'].apply(lambda x: extract_parties(x, party_aliases))

C:\Users\hanna\AppData\Local\Temp\ipykernel_11984\1735183557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_speeches['Mentioned parties'] = text_speeches['Paragraph'].apply(lambda x: extract_parties(x, party_aliases))


In [24]:
text_speeches

,Filename,Period,Date,Item,Speech #,Paragraph #,Speaker,Role,Gender,Party,...,Interjector Party,Verbal interjection,Nonverbal interjection,Interjection type,next_verbal_interjection,next_nonverbal_interjection,next_interjectors,next_interjections,Speaker Paragraph,Mentioned parties
0,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,1,Carsten Schneider,mp,männlich,SPD,...,NaN,False,False,NaN,False,False,[],NaN,NaN,[]
1,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,2,Carsten Schneider,mp,männlich,SPD,...,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN,[]
5,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,3,Carsten Schneider,mp,männlich,SPD,...,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN,[]
9,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,4,Carsten Schneider,mp,männlich,SPD,...,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN,[]
13,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,5,Carsten Schneider,mp,männlich,SPD,...,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'CDU/CSU', 'Lachen', nan)]",NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550753,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,3,Florian Toncar,mp,männlich,FDP,...,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN,[]
550755,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,4,Florian Toncar,mp,männlich,FDP,...,NaN,False,False,NaN,False,False,[],NaN,NaN,[]
550756,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,5,Florian Toncar,mp,männlich,FDP,...,NaN,False,False,NaN,False,True,[],"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN,[]
550758,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,6,Florian Toncar,mp,männlich,FDP,...,NaN,False,False,NaN,False,False,[],NaN,NaN,[]


In [25]:
text_speeches.drop(columns=["Interjector", "Interjector Gender", "Interjector Party",
                 "Verbal interjection", "Nonverbal interjection", "Interjection type", "next_interjectors"], inplace=True)
text_speeches.to_csv('speeches_partymentions_paragraph.csv', index=False)

C:\Users\hanna\AppData\Local\Temp\ipykernel_11984\2315101629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_speeches.drop(columns=["Interjector", "Interjector Gender", "Interjector Party",


In [26]:
text_speeches

,Filename,Period,Date,Item,Speech #,Paragraph #,Speaker,Role,Gender,Party,Paragraph,Interjection,next_verbal_interjection,next_nonverbal_interjection,next_interjections,Speaker Paragraph,Mentioned parties
0,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,1,Carsten Schneider,mp,männlich,SPD,Sehr geehrter Herr Präsident! Sehr geehrte Kol...,False,False,False,NaN,NaN,[]
1,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,2,Carsten Schneider,mp,männlich,SPD,In diesem Haus debattieren wir über die besten...,False,False,True,"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN,[]
5,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,3,Carsten Schneider,mp,männlich,SPD,Der Bundestag muss wieder zur zentralen Bühne ...,False,False,True,"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN,[]
9,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,4,Carsten Schneider,mp,männlich,SPD,"Ihr Politikstil, Frau Merkel, ist ein Grund da...",False,False,True,"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN,[]
13,BT_19_001.xml,19,2017-10-24,Beschlussfassung über die,1,5,Carsten Schneider,mp,männlich,SPD,Sie haben in diesem Wahlkampf jeden politische...,False,False,True,"[('Nonverbal', 'all', 'CDU/CSU', 'Lachen', nan)]",NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550753,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,3,Florian Toncar,mp,männlich,FDP,Dennoch besteht nicht allzu viel Grund zum Sel...,False,False,True,"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN,[]
550755,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,4,Florian Toncar,mp,männlich,FDP,"Der andere Grund, warum ich vor zu viel Euphor...",False,False,False,NaN,NaN,[]
550756,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,5,Florian Toncar,mp,männlich,FDP,"Wenn man sich anschaut, wie dieses Gesetz zust...",False,False,True,"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN,[]
550758,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,6,Florian Toncar,mp,männlich,FDP,"Wir brauchen eine andere Mentalität, einen and...",False,False,False,NaN,NaN,[]


GET LAST 5 DEBATES (PARAGRAPHS)

In [27]:
last_5_filenames = text_speeches['Filename'].drop_duplicates().tail()
last_5_paragraphs = text_speeches[text_speeches['Filename'].isin(last_5_filenames)]
last_5_paragraphs

,Filename,Period,Date,Item,Speech #,Paragraph #,Speaker,Role,Gender,Party,Paragraph,Interjection,next_verbal_interjection,next_nonverbal_interjection,next_interjections,Speaker Paragraph,Mentioned parties
540230,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,Sehr geehrter Herr Präsident! Liebe Kolleginne...,False,False,False,NaN,NaN,[]
540231,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,2,Aydan Özoğuz,mp,weiblich,SPD,Für unsere Bundeswehr schließt sich zunächst e...,False,False,False,NaN,NaN,[]
540232,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,3,Aydan Özoğuz,mp,weiblich,SPD,59 Soldaten haben wir dort verloren. 59 Famili...,False,False,True,"[('Nonverbal', 'all', 'SPD', 'Beifall', nan), ...",NaN,[]
540238,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,4,Aydan Özoğuz,mp,weiblich,SPD,Was bleibt nun nach 20 Jahren Bündnissolidarit...,False,False,False,NaN,NaN,[]
540239,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,5,Aydan Özoğuz,mp,weiblich,SPD,Auch wenn es für uns schwer nachvollziehbar is...,False,False,False,NaN,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550753,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,3,Florian Toncar,mp,männlich,FDP,Dennoch besteht nicht allzu viel Grund zum Sel...,False,False,True,"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN,[]
550755,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,4,Florian Toncar,mp,männlich,FDP,"Der andere Grund, warum ich vor zu viel Euphor...",False,False,False,NaN,NaN,[]
550756,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,5,Florian Toncar,mp,männlich,FDP,"Wenn man sich anschaut, wie dieses Gesetz zust...",False,False,True,"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN,[]
550758,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,6,Florian Toncar,mp,männlich,FDP,"Wir brauchen eine andere Mentalität, einen and...",False,False,False,NaN,NaN,[]


In [28]:
last_5_paragraphs.to_csv('speeches_last_5_partymentions_paragraph.csv', index=False)

GET LAST 5 DEBATES (SENTENCES)

In [29]:
tqdm.pandas()

start = time.time()

print("Step 1: Filtering out interjections...")
text_speeches_df = df[~df['Interjection']]
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 2: Selecting last 5 sessions...")
last_5_filenames = text_speeches_df['Filename'].drop_duplicates().tail()
text_speeches_df = text_speeches_df[text_speeches_df['Filename'].isin(last_5_filenames)]
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 3: Merging paragraphs into speeches...")
text_speeches_df['Speech'] = text_speeches_df.groupby('Speech #')['Paragraph'].transform(' '.join)
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 4: Tokenizing sentences...")
text_speeches_df['Sentences'] = text_speeches_df['Paragraph'].progress_apply(spacy_sent_tokenize)
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 5: Exploding sentences into rows...")
text_speeches_exploded = text_speeches_df.explode("Sentences", ignore_index=True)
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 6: Adding previous and next sentences...")
text_speeches_exploded["prev_sentence"] = text_speeches_exploded.groupby("Speech #")["Sentences"].shift(1)
text_speeches_exploded["next_sentence"] = text_speeches_exploded.groupby("Speech #")["Sentences"].shift(-1)
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 7: Identifying last sentences in paragraphs...")
text_speeches_exploded['is_last_sentence'] = text_speeches_exploded.groupby('Paragraph').cumcount(ascending=False) == 0
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 8: Adjusting interjection columns...")
text_speeches_exploded['next_verbal_interjection'] = text_speeches_exploded['is_last_sentence'] * text_speeches_exploded['next_verbal_interjection']
text_speeches_exploded['next_nonverbal_interjection'] = text_speeches_exploded['is_last_sentence'] * text_speeches_exploded['next_nonverbal_interjection']
text_speeches_exploded['next_verbal_interjection'] = text_speeches_exploded.groupby('Paragraph')['next_verbal_interjection'].ffill()
text_speeches_exploded['next_nonverbal_interjection'] = text_speeches_exploded.groupby('Paragraph')['next_nonverbal_interjection'].ffill()
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 9: Forward filling next interjections...")
text_speeches_exploded['next_interjections'] = text_speeches_exploded['is_last_sentence'] * text_speeches_exploded['next_interjections']
text_speeches_exploded['next_interjections'] = text_speeches_exploded.groupby('Paragraph')['next_interjections'].ffill()
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 10: Filling missing values...")
text_speeches_exploded["prev_sentence"] = text_speeches_exploded["prev_sentence"].fillna("")
text_speeches_exploded["Sentences"] = text_speeches_exploded["Sentences"].fillna("")
text_speeches_exploded["next_sentence"] = text_speeches_exploded["next_sentence"].fillna("")
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

start = time.time()
print("Step 11: Extracting mentioned parties...")
text_speeches_exploded['Mentioned parties'] = text_speeches_exploded['Sentences'].progress_apply(lambda x: extract_parties(x, party_aliases))
print(f"Done! Took {time.time() - start:.2f} seconds.\n")

print("Processing complete! 🎉")


Step 1: Filtering out interjections...
Done! Took 0.46 seconds.

Step 2: Selecting last 5 sessions...
Done! Took 0.06 seconds.

Step 3: Merging paragraphs into speeches...
Done! Took 0.11 seconds.

Step 4: Tokenizing sentences...


100%|██████████| 5523/5523 [00:24<00:00, 222.64it/s]


Done! Took 24.82 seconds.

Step 5: Exploding sentences into rows...
Done! Took 0.04 seconds.

Step 6: Adding previous and next sentences...
Done! Took 0.01 seconds.

Step 7: Identifying last sentences in paragraphs...
Done! Took 0.16 seconds.

Step 8: Adjusting interjection columns...
Done! Took 0.09 seconds.

Step 9: Forward filling next interjections...
Done! Took 0.04 seconds.

Step 10: Filling missing values...
Done! Took 0.02 seconds.

Step 11: Extracting mentioned parties...


100%|██████████| 18609/18609 [00:53<00:00, 349.06it/s]


Done! Took 53.32 seconds.

Processing complete! 🎉


In [30]:
text_speeches_exploded

,Filename,Period,Date,Item,Speech #,Paragraph #,Speaker,Role,Gender,Party,...,next_nonverbal_interjection,next_interjectors,next_interjections,Speaker Paragraph,Speech,Sentences,prev_sentence,next_sentence,is_last_sentence,Mentioned parties
0,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Sehr geehrter Herr Präsident!,,Liebe Kolleginnen und Kollegen!,False,[]
1,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Liebe Kolleginnen und Kollegen!,Sehr geehrter Herr Präsident!,Mit dem Abzug der NATO-Truppen nach zwei Jahrz...,False,[]
2,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Mit dem Abzug der NATO-Truppen nach zwei Jahrz...,Liebe Kolleginnen und Kollegen!,Und der Ausgang der Verhandlungen ist weiterhi...,False,[]
3,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Und der Ausgang der Verhandlungen ist weiterhi...,Mit dem Abzug der NATO-Truppen nach zwei Jahrz...,Für unsere Bundeswehr schließt sich zunächst e...,True,[]
4,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,2,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Für unsere Bundeswehr schließt sich zunächst e...,Und der Ausgang der Verhandlungen ist weiterhi...,59 Soldaten haben wir dort verloren.,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18604,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,5,Florian Toncar,mp,männlich,FDP,...,False,[],,NaN,Vielen Dank. – Frau Präsidentin! Liebe Kollegi...,"Wenn man sich anschaut, wie dieses Gesetz zust...",Sie haben sich jetzt mit Ach und Krach über di...,"Ich glaube nicht, dass man so arbeiten kann, w...",False,[]
18605,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,5,Florian Toncar,mp,männlich,FDP,...,True,[],"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN,Vielen Dank. – Frau Präsidentin! Liebe Kollegi...,"Ich glaube nicht, dass man so arbeiten kann, w...","Wenn man sich anschaut, wie dieses Gesetz zust...","Wir brauchen eine andere Mentalität, einen and...",True,[]
18606,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,6,Florian Toncar,mp,männlich,FDP,...,False,[],NaN,NaN,Vielen Dank. – Frau Präsidentin! Liebe Kollegi...,"Wir brauchen eine andere Mentalität, einen and...","Ich glaube nicht, dass man so arbeiten kann, w...",Das wird sich hoffentlich nach dem 26. Septemb...,False,[]
18607,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,6,Florian Toncar,mp,männlich,FDP,...,False,[],NaN,NaN,Vielen Dank. – Frau Präsidentin! Liebe Kollegi...,Das wird sich hoffentlich nach dem 26. Septemb...,"Wir brauchen eine andere Mentalität, einen and...",Herzlichen Dank.,True,[]


In [31]:
text_speeches_exploded.to_csv('speeches_last_5_partymentions_sentence.csv', index=False)

In [32]:
text_speeches_exploded

,Filename,Period,Date,Item,Speech #,Paragraph #,Speaker,Role,Gender,Party,...,next_nonverbal_interjection,next_interjectors,next_interjections,Speaker Paragraph,Speech,Sentences,prev_sentence,next_sentence,is_last_sentence,Mentioned parties
0,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Sehr geehrter Herr Präsident!,,Liebe Kolleginnen und Kollegen!,False,[]
1,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Liebe Kolleginnen und Kollegen!,Sehr geehrter Herr Präsident!,Mit dem Abzug der NATO-Truppen nach zwei Jahrz...,False,[]
2,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Mit dem Abzug der NATO-Truppen nach zwei Jahrz...,Liebe Kolleginnen und Kollegen!,Und der Ausgang der Verhandlungen ist weiterhi...,False,[]
3,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,1,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Und der Ausgang der Verhandlungen ist weiterhi...,Mit dem Abzug der NATO-Truppen nach zwei Jahrz...,Für unsere Bundeswehr schließt sich zunächst e...,True,[]
4,BT_19_235.xml,19,2021-06-23,Aktuelle Stunde zum geordneten Rückzug der NAT...,32489,2,Aydan Özoğuz,mp,weiblich,SPD,...,False,[],NaN,NaN,Sehr geehrter Herr Präsident! Liebe Kolleginne...,Für unsere Bundeswehr schließt sich zunächst e...,Und der Ausgang der Verhandlungen ist weiterhi...,59 Soldaten haben wir dort verloren.,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18604,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,5,Florian Toncar,mp,männlich,FDP,...,False,[],,NaN,Vielen Dank. – Frau Präsidentin! Liebe Kollegi...,"Wenn man sich anschaut, wie dieses Gesetz zust...",Sie haben sich jetzt mit Ach und Krach über di...,"Ich glaube nicht, dass man so arbeiten kann, w...",False,[]
18605,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,5,Florian Toncar,mp,männlich,FDP,...,True,[],"[('Nonverbal', 'all', 'FDP', 'Beifall', nan)]",NaN,Vielen Dank. – Frau Präsidentin! Liebe Kollegi...,"Ich glaube nicht, dass man so arbeiten kann, w...","Wenn man sich anschaut, wie dieses Gesetz zust...","Wir brauchen eine andere Mentalität, einen and...",True,[]
18606,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,6,Florian Toncar,mp,männlich,FDP,...,False,[],NaN,NaN,Vielen Dank. – Frau Präsidentin! Liebe Kollegi...,"Wir brauchen eine andere Mentalität, einen and...","Ich glaube nicht, dass man so arbeiten kann, w...",Das wird sich hoffentlich nach dem 26. Septemb...,False,[]
18607,BT_19_239.xml,19,2021-09-07,Beratung der Beschlussempfehlung des Ausschuss...,33060,6,Florian Toncar,mp,männlich,FDP,...,False,[],NaN,NaN,Vielen Dank. – Frau Präsidentin! Liebe Kollegi...,Das wird sich hoffentlich nach dem 26. Septemb...,"Wir brauchen eine andere Mentalität, einen and...",Herzlichen Dank.,True,[]
